In [6]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
data = pd.read_excel('Pro_Data.xlsx')
import re
import jieba
jieba.load_userdict("./Pro_dict.txt")


def chinese_word_cut(mytext):
    #jieba.load_userdict('dic.txt')  # 这里你可以添加jieba库识别不了的网络新词，避免将一些新词拆开
    jieba.initialize()
    # 文本预处理 ：去除一些无用的字符只提取出中文出来
    new_data = re.findall('[\u4e00-\u9fa5]+', str(mytext), re.S)
    new_data = " ".join(new_data)

    # 文本分词
    seg_list_exact = jieba.cut(new_data, cut_all=True)
    result_list = []
    with open('Prostop.txt','r', encoding='utf-8') as f:  # 可根据需要打开停用词库，然后加上不想显示的词语
        con = f.readlines()
        stop_words = set()
        for i in con:
            i = i.replace("\n", "")  # 去掉读取每一行数据的\n
            stop_words.add(i)

    for word in seg_list_exact:
        if word not in stop_words and len(word) > 1:
            result_list.append(word)
    return " ".join(result_list)


data["report_cutted"] = data.report.apply(chinese_word_cut)
#文本向量化
from sklearn.feature_extraction.text import CountVectorizer

n_features = 1000  # 提取1000个特征词语
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=20)
tf = tf_vectorizer.fit_transform(data.report_cutted)
#构建LDA
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 9
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=200,
                                learning_method='batch',
                                learning_offset=50,
                                doc_topic_prior=0.1,
                                topic_word_prior=0.01,
                               random_state=10000000)  # 关于模型的参数,可查看官方文档
lda.fit(tf)
#输出每个主题对应词语
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:" )
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword
# 输出每个主题对应词语
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
公路 农村公路 交通 养护 运输 交通运输 物流 邮政 客运 寄递 快递 交通安全 补助 公交 道路交通 公安 车辆 旅游 执法 路网
Topic #1:
农牧 农业产业 龙头企业 家庭 农场 蔬菜 产业化 农牧业 农户 产业园 牧业 贷款 食品 农技 信贷 特色产业 银行 补助 物流 销售
Topic #2:
粮食 应急 执法 储备 食品 检查 灾害 违法 救援 产品质量 预警 市场监管 粮油 事故 防汛 处置 法治 抗旱 疫情 隐患
Topic #3:
创业 就业 科普 学校 文化 培养 家庭 医疗 儿童 养老 民族 卫生 科研 体育 生活 高校 教师 公共服务 学生 院校
Topic #4:
农机 机械 机械化 装备 种植 秸秆 补贴 作业 农业机械 玉米 水稻 粮食 品种 农作物 农药 病虫 小麦 机具 施肥 蔬菜
Topic #5:
农田 脱贫 高标准 耕地 高标准农田 帮扶 粮食 扶贫 管护 贫困 收入 整治 生活 基本农田 宅基地 美丽 搬迁 收益 改善 产能
Topic #6:
旅游 数字 文化 物流 消费 智慧 数字化 供应 交通 互联网 公共服务 传统 电子 装备 供应链 服务业 智能化 高端 产业园 业态
Topic #7:
污染 生态环境 用水 供水 水资源 节水 修复 生活 污水 水源 河流 自然资源 防治 整治 排放 管控 环境保护 水库 湿地 垃圾
Topic #8:
养殖 畜禽 畜牧 种业 品种 水产 渔业 种质 生猪 动物 牧业 畜牧业 屠宰 育种 养殖场 粪污 良种 种子 疫病 生物
